In [4]:
from datasets import load_dataset
import re

/Users/dtemnov/Projects/comma_placement/.venv/lib/python3.11/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


# Get WikiText

In [5]:
dataset = load_dataset("wikitext", 'wikitext-103-v1')

/Users/dtemnov/Projects/comma_placement/.venv/lib/python3.11/site-packages/datasets/table.py:1421: FutureWarning: promote has been superseded by mode='default'.
  table = cls._concat_blocks(blocks, axis=0)


## Prepare dataset

Wikitext contains a lot of delimiters and empty lines, so we need to clean it up a little bit.

In [53]:
space_remover = r'\s([,?.!:;](?:\s|$))'
quote_space_remover = r'"\s*([^"]*?)\s*"'

def remove_spaces(samples):
    samples = [sample.strip() for sample in samples]
    samples = [re.sub(space_remover, r'\1', sample) for sample in samples]
    samples = [re.sub(quote_space_remover, r'"\1"', sample) for sample in samples]
    samples = [sample.replace(" '", "'") for sample in samples]
    return samples

def remove_empty(samples: list) -> list:
    samples = [sample for sample in samples if sample]
    return samples

def remove_titles(samples: list) -> list:
    samples = [sample for sample in samples if "=" not in sample]
    return samples

def remove_short(samples: list) -> list:
    samples = [sample for sample in samples if len(sample) > 300]
    return samples

def preprocess_texts(samples: list) -> list:
    samples = remove_titles(samples)
    samples = remove_spaces(samples)
    samples = remove_empty(samples)
    samples = remove_short(samples)
    return samples

In [54]:
dataset_train = dataset["train"][0:100000]["text"]

In [ ]:
dataset_train[:10]

In [56]:
samples = preprocess_texts(dataset_train)

In [57]:
len(samples)

35204

The final total size of the dataset is about 35k samples. This amount of data should be enough for training and validation for this pretty simple task.

In [58]:
with open("../data/raw/raw_wiki_lines.txt", "w") as f:
    f.writelines([sample+"\n" for sample in samples])